In [48]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import numpy as np
from keras.utils import np_utils
import re

In [57]:
# count = 1

# with open ('./europarl-corpus/train_small.txt', 'r') as f:
#     if (count < 5000):
#         train_eu = f.readline()
#         count=count+1




# # # concatenate all sentences in train_eu
# # train_eu = train_eu.split('\n')
# # train_eu = ' '.join(train_eu)
# # train_eu = train_eu.split('.')
# # train_eu = ' '.join(train_eu)
# # train_eu = train_eu.split(' ')
# # train_eu = ' '.join(train_eu)
# # remove punctuation from train_eu
# train_eu = re.sub(r'[^\w\s]','',train_eu)

# # select first 10000 sentences from train.europarl for training
# # train_eu = train_eu.split('\n')
# # train_eu = train_eu[:100000]



# # train_eu

# # # store the first 10000 sentences in a variable
# # train_eu = train_eu.split('\n')
# # train_eu = train_eu[:10000]

# # train_eu

# # # convert list to string
# # train_eu = ' '.join(train_eu)


# # store train_eu in a text file
# with open ('./europarl-corpus/final_train.txt', 'w') as f:
#     f.write(train_eu)

# train_eu


with open('./europarl-corpus/final_train.txt', 'r') as f:
    train_eu = f.read()

# select the first 10000 sentences from train_eu
train_eu = train_eu.split('.')
train_eu = train_eu[:10000]

str = ''
for i in train_eu:
    str = str + i + '.'


with open ('./europarl-corpus/final_train.txt', 'w') as f:
    f.write(str)


In [36]:
tokenizer = Tokenizer()
data = train_eu

corpus = data.lower().split("\n")
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
# print(total_words)
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = np_utils.to_categorical(label, num_classes=total_words)




[[  0   0   0 ...   0   0 306]
 [  0   0   0 ...   0 306   3]
 [  0   0   0 ... 306   3   1]
 ...
 [  0   0   0 ... 284   4 302]
 [  0   0   0 ...   4 302   3]
 [  0   0   0 ... 302   3   8]] [[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
input_len = max_sequence_len - 1    model = Sequential()
model.add(Embedding(total_words, 10, input_length=input_len))
model.add(LSTM(150))
model.add(Dropout(0.1))
model.add(Dense(total_words, activation='softmax'))    model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(predictors, label, epochs=100, verbose=1)